In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from math import sqrt
from datetime import datetime

%matplotlib inline

In [2]:
# change print option to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

# increase size of plots
sns.set(rc={'figure.figsize':(12,10)})

In [3]:
confirmed_raw = pd.read_csv('data/time_series_covid19_confirmed_global.csv')
deaths_raw = pd.read_csv('data/time_series_covid19_deaths_global.csv')
recovered_raw = pd.read_csv('data/time_series_covid19_recovered_global.csv')

In [4]:
print("Confimed case file size: ", confirmed_raw.shape)
print("Death case file size: ", deaths_raw.shape)
print("Recovered case file size: ", recovered_raw.shape)

Confimed case file size:  (273, 385)
Death case file size:  (273, 385)
Recovered case file size:  (258, 385)


In [5]:
confirmed_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,54854,54891,54939,55008,55023,55059,55121,55174,55231,55265
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,74567,75454,76350,77251,78127,78992,79934,80941,81993,83082
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,106359,106610,106887,107122,107339,107578,107841,108116,108381,108629
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,9716,9779,9837,9885,9937,9972,10017,10070,10137,10172
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,19580,19672,19723,19782,19796,19829,19900,19937,19996,20030


Next we will check if there is any missing values in the matrix

In [6]:
confirmed_raw.isnull().sum()

Province/State    189
Country/Region      0
Lat                 1
Long                1
1/22/20             0
                 ... 
2/1/21              0
2/2/21              0
2/3/21              0
2/4/21              0
2/5/21              0
Length: 385, dtype: int64

In [7]:
deaths_raw.isnull().sum()

Province/State    189
Country/Region      0
Lat                 1
Long                1
1/22/20             0
                 ... 
2/1/21              0
2/2/21              0
2/3/21              0
2/4/21              0
2/5/21              0
Length: 385, dtype: int64

In [8]:
recovered_raw.isnull().sum()

Province/State    190
Country/Region      0
Lat                 0
Long                0
1/22/20             0
                 ... 
2/1/21              0
2/2/21              0
2/3/21              0
2/4/21              0
2/5/21              0
Length: 385, dtype: int64

As shown above,  we know that there is one object has missing long/lat informations, this may cause a problem later when we try to use the information to map out the data.

So we will just remove entry for our case.

In [9]:
# remove missing values
confirmed_raw = confirmed_raw[confirmed_raw['Lat'].notna()]
deaths_raw = deaths_raw[deaths_raw['Lat'].notna()]

confirmed_raw.isnull().sum()

Province/State    189
Country/Region      0
Lat                 0
Long                0
1/22/20             0
                 ... 
2/1/21              0
2/2/21              0
2/3/21              0
2/4/21              0
2/5/21              0
Length: 385, dtype: int64

In [10]:
confirmed_raw['Province/State'].fillna(confirmed_raw['Country/Region'], inplace=True)
deaths_raw['Province/State'].fillna(deaths_raw['Country/Region'], inplace=True)
recovered_raw['Province/State'].fillna(recovered_raw['Country/Region'], inplace=True)

In [11]:
confirmed_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
0,Afghanistan,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,54854,54891,54939,55008,55023,55059,55121,55174,55231,55265
1,Albania,Albania,41.15330,20.168300,0,0,0,0,0,0,...,74567,75454,76350,77251,78127,78992,79934,80941,81993,83082
2,Algeria,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,106359,106610,106887,107122,107339,107578,107841,108116,108381,108629
3,Andorra,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,9716,9779,9837,9885,9937,9972,10017,10070,10137,10172
4,Angola,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,19580,19672,19723,19782,19796,19829,19900,19937,19996,20030


Next we will calculate daily cases (daily confirm cases, daily new deaths, daily recovered)

Since the data is collected with total cases everyday, how we can get the daily count is to use the following formula: 

$$ DailyNewCases = TodyTotalCases - YesterdayTotalCases $$

In [12]:
def calcDailyCases(ori_mtx):
    str_col = 4
    conv_mtx = ori_mtx.copy()
    ori_mtx_raw = ori_mtx.iloc[:, str_col:].copy().to_numpy()
    
    # subtract from previous day to get the exact daily count
    ori_mtx_raw_daily = ori_mtx_raw[:, 1:,] - ori_mtx_raw[:, :-1]
    
    # when subtracting, first row will be lost in the process we need to extract from the orignal matrix
    #starting_date = ori_mtx[:,0].copy()
    
    # then we will append back to the processed matrix
    #numpy_confirmed_raw_daily = np.insert(numpy_confirmed_raw_daily, 0, starting_date, axis=1)
    
    # first day report can be retrived from the original matrix
    conv_mtx.iloc[:, (str_col+1):] = ori_mtx_raw_daily
    
    return conv_mtx

In [13]:
confirmed_daily = calcDailyCases(confirmed_raw)
deaths_daily = calcDailyCases(deaths_raw)
recovered_daily = calcDailyCases(recovered_raw)

In [14]:
confirmed_daily.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
0,Afghanistan,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,104,37,48,69,15,36,62,53,57,34
1,Albania,Albania,41.15330,20.168300,0,0,0,0,0,0,...,876,887,896,901,876,865,942,1007,1052,1089
2,Algeria,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,262,251,277,235,217,239,263,275,265,248
3,Andorra,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,78,63,58,48,52,35,45,53,67,35
4,Angola,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,27,92,51,59,14,33,71,37,59,34


In [15]:
deaths_daily.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
0,Afghanistan,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,8,2,1,0,4,1,1,1,0
1,Albania,Albania,41.15330,20.168300,0,0,0,0,0,0,...,7,11,8,11,11,13,5,6,14,15
2,Algeria,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6,4,3,4,3,3,4,2,4,5
3,Andorra,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,1,2,0,1,0,0,1,1,0,1
4,Angola,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,0,2,0,0,2,0,2,2,3,1


In [16]:
recovered_daily.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
0,Afghanistan,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,90,34,23,3,70,44,75,184,13,23
1,Albania,Albania,41.15330,20.168300,0,0,0,0,0,0,...,503,523,514,507,497,498,455,581,585,676
2,Algeria,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,202,207,211,196,192,186,202,208,196,194
3,Andorra,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,77,91,44,59,48,113,46,61,82,49
4,Angola,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,214,90,226,97,20,145,52,103,46,50


In [17]:
recovered_daily[recovered_daily['Country/Region'] == 'Malaysia']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21
159,Malaysia,Malaysia,4.210484,101.975766,0,0,0,0,0,0,...,1858,3281,3423,3805,4522,4280,3661,3804,4092,3392


Now we will save the converted data and visualize with power of BI

In [20]:
confirmed_daily.to_csv('conv_confirmed.csv', index=False)
deaths_daily.to_csv('conv_deaths.csv', index=False)
recovered_daily.to_csv('conv_recovered.csv', index=False)